In [24]:
# 自动将matplotlib绘图嵌入到notebook中
%matplotlib inline

#自动补全
%config IPCompleter.greedy=True

# 设置默认的matplotlib绘图比例
%pylab inline
pylab.rcParams['figure.figsize'] = (15, 9) 

Populating the interactive namespace from numpy and matplotlib


In [86]:
from pymongo import MongoClient
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import DataFrame
from functools import reduce
import datetime

In [26]:
client = MongoClient('127.0.0.1', 27017, username='mjh', password='666666', authSource='stock')
db = client['stock']
db.list_collection_names()

['trade_cal',
 'fina_indicator',
 'daily_basic',
 'cashflow',
 'daily',
 'stock_basic',
 'income',
 'dividend',
 'balancesheet',
 'stock_company']

In [27]:
daily = DataFrame(
    list(db['daily'].find({'ts_code': '000001.SZ'})),
    columns=['ts_code', 'trade_date', 'open', 'high', 'low', 'close',
             'pre_close', 'change', 'pct_chg', 'vol', 'amount'])
daily.head()

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,000001.SZ,20011022,12.68,13.09,12.58,12.81,12.72,0.09,0.71,56638.87,73022.6534
1,000001.SZ,20011023,14.09,14.09,13.50,14.09,12.81,1.28,9.99,288159.18,403209.4207
2,000001.SZ,20011024,14.33,14.50,13.81,14.11,14.09,0.02,0.14,269604.58,382937.6694
3,000001.SZ,20011025,14.00,14.09,13.75,13.78,14.11,-0.33,-2.34,83517.58,115800.0658
4,000001.SZ,20011026,13.78,14.00,13.66,13.90,13.78,0.12,0.87,52168.97,72146.0778


In [28]:
data = daily[daily['trade_date'] > '20180901']
data.head()

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
3936,000001.SZ,20180903,10.09,10.30,10.09,10.19,10.13,0.06,0.5923,711695.67,725303.291
3937,000001.SZ,20180904,10.20,10.55,10.07,10.43,10.19,0.24,2.3553,1296342.08,1339756.563
3938,000001.SZ,20180905,10.30,10.33,10.05,10.05,10.43,-0.38,-3.6433,1179453.17,1202345.697
3939,000001.SZ,20180906,10.05,10.14,9.95,9.97,10.05,-0.08,-0.7960,711777.55,713975.749
3940,000001.SZ,20180907,10.04,10.18,9.93,10.01,9.97,0.04,0.4012,809035.34,815151.722


In [82]:
!cat chart.py

#!/usr/bin/env python
# -*- coding: utf-8 -*-

import os
import sys
import datetime
import numpy as np

# MatPlotLib 的主要模組
import matplotlib.pyplot as plt

# 畫圖形週邊東西的套件
from matplotlib import gridspec
from matplotlib.ticker import FuncFormatter

import bisect

# 畫圖用的套件
from matplotlib.collections import LineCollection, PolyCollection
from matplotlib.colors import colorConverter
from matplotlib.lines import Line2D
from matplotlib.patches import Rectangle
from matplotlib import colors as mcolors

from dateutil.parser import parse

# 讀股價歷史資料
#import pandas_datareader.data as web

import pandas as pd

has_chinese_font = False
chinese_font_path = r"c:\windows\fonts\simsun.ttc"
if os.path.exists(chinese_font_path):
    from matplotlib.font_manager import FontProperties
    font = FontProperties(fname=chinese_font_path, size=24)
    has_chinese_font = True


def candlestick(ax,
                opens,
                highs,
                lows,
                closes,
                width=4,

In [29]:
def candlestick(ax,
                opens,
                highs,
                lows,
                closes,
                width=4,
                colorup='g',
                colordown='r',
                alpha=0.75):
    '''绘制K线图'''

    delta = width / 2.

    # 中间的Box
    barVerts = [((i - delta, open), (i - delta, close), (i + delta, close),
                 (i + delta, open))
                for i, open, close in zip(range(len(opens)), opens, closes)]

    # 下影线
    downSegments = [((i, low), (i, min(open, close)))
                    for i, low, high, open, close in zip(
                        range(len(lows)), lows, highs, opens, closes)]

    # 上影线
    upSegments = [((i, max(open, close)), (i, high))
                  for i, low, high, open, close in zip(
                      range(len(lows)), lows, highs, opens, closes)]

    rangeSegments = upSegments + downSegments

    r, g, b = colorConverter.to_rgb(colorup)
    colorup = r, g, b, alpha
    r, g, b = colorConverter.to_rgb(colordown)
    colordown = r, g, b, alpha
    colord = {
        True: colorup,
        False: colordown,
    }
    colors = [colord[open < close] for open, close in zip(opens, closes)]

    useAA = 0,  # use tuple here
    lw = 0.5,  # and here
    rangeCollection = LineCollection(
        rangeSegments,
        colors=((0, 0, 0, 1), ),
        linewidths=lw,
        antialiaseds=useAA,
    )

    barCollection = PolyCollection(
        barVerts,
        facecolors=colors,
        edgecolors=((0, 0, 0, 1), ),
        antialiaseds=useAA,
        linewidths=lw,
    )

    minx, maxx = 0, len(rangeSegments) / 2
    miny = min([low for low in lows])
    maxy = max([high for high in highs])

    corners = (minx, miny), (maxx, maxy)
    ax.update_datalim(corners)
    ax.autoscale_view()

    # add these last
    ax.add_collection(rangeCollection)
    ax.add_collection(barCollection)

    return rangeCollection, barCollection

In [30]:
def volume_overlay(ax,
                   opens,
                   closes,
                   volumes,
                   colorup='g',
                   colordown='r',
                   width=4,
                   alpha=1.0):
    """Add a volume overlay to the current axes.  The opens and closes
    are used to determine the color of the bar.  -1 is missing.  If a
    value is missing on one it must be missing on all

    Parameters
    ----------
    ax : `Axes`
        an Axes instance to plot to
    opens : sequence
        a sequence of opens
    closes : sequence
        a sequence of closes
    volumes : sequence
        a sequence of volumes
    width : int
        the bar width in points
    colorup : color
        the color of the lines where close >= open
    colordown : color
        the color of the lines where close <  open
    alpha : float
        bar transparency

    Returns
    -------
    ret : `barCollection`
        The `barrCollection` added to the axes

    """

    colorup = mcolors.to_rgba(colorup, alpha)
    colordown = mcolors.to_rgba(colordown, alpha)
    colord = {True: colorup, False: colordown}
    colors = [
        colord[open < close] for open, close in zip(opens, closes)
        if open != -1 and close != -1
    ]

    delta = width / 2.
    bars = [((i - delta, 0), (i - delta, v), (i + delta, v), (i + delta, 0))
            for i, v in enumerate(volumes) if v != -1]

    barCollection = PolyCollection(
        bars,
        facecolors=colors,
        edgecolors=((0, 0, 0, 1), ),
        antialiaseds=(0, ),
        linewidths=(0.5, ),
    )

    ax.add_collection(barCollection)
    corners = (0, 0), (len(bars), max(volumes))
    ax.update_datalim(corners)
    ax.autoscale_view()

    # add these last
    return barCollection

In [98]:
def millions(x, pos):
    'The two args are the value and tick position'
    return '%1.1fM' % (x * 1e-6)

def thousands(x, pos):
    'The two args are the value and tick position'
    return '%1.1fK' % (x * 1e-3)

def getListOfDates(start_date, end_date):
    '''获取月份的第一天'''
    dates = [
        datetime.date(m // 12, m % 12 + 1, 1)
        for m in range(start_date.year * 12 + startdate.month -
                       1, enddate.year * 12 + enddate.month)
    ]
    return np.array(dates)

def getDateIndex(dates, tickdates):
    "找出最接近 tickdate 的日期的 index"
    index = [bisect.bisect_left(dates, tickdate) for tickdate in tickdates]
    return np.array(index)

def getMonthNames(dates, index):
    "取得 X 軸上面日期的表示方式"
    names = []
    for i in index:
        if i == 0:
            if dates[i].day > 15:
                names.append("")
            else:
                names.append(dates[i].strftime("%b'%y"))
        else:
            names.append(dates[i].strftime("%b'%y"))

    return np.array(names)

In [99]:
def drawStockPanel(data, title='', colorup='g', colordown='r'):
    '''绘制股票面板，包含K线图、成交量图和MACD图'''

    dates = data['dates']
    opens = data['opens']
    closes = data['closes']
    lows = data['lows']
    highs = data['highs']
    volumes = data['volumes']
    amounts = data['amounts']
    
    dates = dates if isinstance(dates, list) else list(dates)
    opens = opens if isinstance(opens, list) else list(opens)
    closes = closes if isinstance(closes, list) else list(closes)
    lows = lows if isinstance(lows, list) else list(lows)
    highs = highs if isinstance(highs, list) else list(highs)
    volumes = volumes if isinstance(volumes, list) else list(volumes)
    amounts = amounts if isinstance(amounts, list) else list(amounts)

    if not reduce(lambda x, y: x and y,
           map(lambda x: x == len(dates),
               [len(opens), len(closes), len(lows), len(highs),
                len(volumes), len(amounts)])):
        raise ValueError('数据长度不相同')

    if len(dates) == 0:
        raise ValueError('空数据')
        
    start_date = dates[0]
    end_date = dates[-1]
  
    tickdates = getListOfDates(start_date, end_date)
    tickindex = getDateIndex(dates, tickdates)
    ticknames = getMonthNames(dates, tickindex)

    millionformatter = FuncFormatter(millions)
    thousandformatter = FuncFormatter(thousands)

    fig = plt.figure(figsize=(16, 12))
    fig.subplots_adjust(bottom=0.1)
    fig.subplots_adjust(hspace=0)

    gs = gridspec.GridSpec(2, 1, height_ratios=[4, 1])

    ax0 = plt.subplot(gs[0])
    candles = candlestick(
        ax0,
        df.Open,
        df.High,
        df.Low,
        df.Close,
        width=1,
        colorup=colorup,
        colordown=colordown)

    last_price = "Date:{}, Open:{}, High:{}, Low:{}, Close:{}, Volume:{}".format(
        df.Date[-1], df.Open[-1], df.High[-1], df.Low[-1], df.Close[-1],
        df.Volume[-1])
    ax0.text(
        0.99,
        0.97,
        last_price,
        horizontalalignment='right',
        verticalalignment='bottom',
        transform=ax0.transAxes)

    draw_price_ta(ax0, df)

    def format_coord1(x, y):
        "用來顯示股價相關資訊"
        try:
            index = int(x + 0.5)
            if index < 0 or index >= len(df.Date):
                return ""
            else:
                return 'x=%s, y=%1.1f, price=%1.1f' % (
                    df.Date[int(x + 0.5)], y, df.Close[int(x + 0.5)])
        except Exception as e:
            print(e.args)
            return ''

    def format_coord2(x, y):
        "用來顯示 Volume 的相關資訊"
        try:
            index = int(x + 0.5)
            if index < 0 or index >= len(df.Date):
                return ""
            else:
                return 'x=%s, y=%1.1fM, volume=%1.1fM' % (df.Date[int(
                    x + 0.5)], y * 1e-6, df.Volume[int(x + 0.5)] * 1e-6)
        except Exception as e:
            print(e.args)
            return ''

    ax0.set_xticks(tickindex)
    ax0.set_xticklabels(ticknames)
    ax0.format_coord = format_coord1
    ax0.legend(loc='upper left', shadow=True, fancybox=True)
    ax0.set_ylabel('Price($)', fontsize=16)
    if has_chinese_font:
        ax0.set_title(
            title, fontsize=24, fontweight='bold', fontproperties=font)
    else:
        ax0.set_title(title, fontsize=24, fontweight='bold')
    ax0.grid(True)

    ax1 = plt.subplot(gs[1], sharex=ax0)
    vc = volume_overlay(
        ax1,
        df.Open,
        df.Close,
        df.Volume,
        colorup=colorup,
        colordown=colordown,
        width=1)

    ax1.set_xticks(tickindex)
    ax1.set_xticklabels(ticknames)
    ax1.format_coord = format_coord2

    ax1.tick_params(axis='x', direction='out', length=5)
    ax1.yaxis.set_major_formatter(millionformatter)
    ax1.yaxis.tick_right()
    ax1.yaxis.set_label_position("right")
    ax1.set_ylabel('Volume', fontsize=16)
    ax1.grid(True)

    plt.setp(ax0.get_xticklabels(), visible=False)

    cursor0 = Cursor(ax0)
    cursor1 = Cursor(ax1)
    plt.connect('motion_notify_event', cursor0.mouse_move)
    plt.connect('motion_notify_event', cursor1.mouse_move)

    # 使用 cursor 的時候，如果沒有設定上下限，圖形的上下限會跑掉
    yh = df.High.max()
    yl = df.Low.min()
    ax0.set_ylim(yl - (yh - yl) / 20.0, yh + (yh - yl) / 20.0)
    ax0.set_xlim(0, len(df.Date) - 1)

    plt.show()


In [100]:
drawStockPanel({
    'dates': pd.to_datetime(data['trade_date']),
    'opens': data['open'],
    'closes': data['close'],
    'lows': data['low'],
    'highs': data['high'],
    'volumes': data['vol'],
    'amounts': data['amount'],
})

<class 'list'>


NameError: name 'startdate' is not defined